In [ ]:
import os
import json

from library_dicom.dicom_processor.tools.folders import *
from library_dicom.dicom_processor.tools.series import get_series_object

import csv
from library_dicom.dicom_processor.model.Series import Series

#### - Get every serie paths and generate JSON file 

In [ ]:
series_paths = get_series_path('abs//path//Validated_Dicom//directory')
export_folder = 'abs//path//json_export//directory'

In [ ]:
index_problem = []
for serie_path in series_paths:
    print(series_paths.index(serie_path))
    try:
        dicom_serie = get_series_object(serie_path)
        dicomsInfo = dicom_serie.get_series_details()
        write_json_file(export_folder, dicomsInfo['series']['SeriesInstanceUID'], dicomsInfo)
    except Exception as err:
        print(err)
        print(dicomsInfo)
        index_problem.append(series_paths.index(serie_path))

In [ ]:
list_dir_json = os.listdir(export_folder)
dataset = []
for file_ in list_dir_json : 
    dataset.append([os.path.join(json_path, file_)])

#### - Get informations from every json in list 

In [ ]:
for patient in dataset : 
    with open(patient[0]) as json_file : 
        reader = json.load(json_file)
        patient_id = reader["patient"]["PatientID"]
        study_uid = reader["study"]['StudyInstanceUID']
        modal = reader['series']['Modality']
        path = reader['path']
        type_ = reader['study']["StudyDescription"] #has to be PET0, PET2 for example
        patient.append(path)
        patient.append(modal)
        patient.append(type_)
        patient.append(patient_id)
        patient.append(study_uid)

#each patient : [json_file, path of the serie, modality of the serie, type_, patient_id, study_uid] value

#### - Reunite CT/PT series with same study uid 

In [ ]:
#get every study_uid 
study_uid = []
for patient in dataset :
    if patient[-1] not in study_uid : 
        study_uid.append(patient[-1]) 
print('Number of study_uid :', len(study_uid))

reunited_series = []
for uid in study_uid : 
    sub = []
    for patient in dataset : 
        if if uid in patient : 
            sub.append(patient)
    reunited_series.append(sub)
#reunited_series = [ [uid 1 : [serie_1], [serie_2]], [uid 2 : [serie_1], [serie_2]], ...]
dataset = []
for study in reunited_series : 
    if len(study) == 2 : #a CT and PET series
        study[0].append(study[1][0])#json_path 
        study[0].append(study[1][1])#path 
        study[0].append(study[1][2])#modal 
    dataset.append(study[0])
#dataset = [ [json_path_1, path_serie_1, modal_1, type_1, patient_id, study_uid, json_path_2, path_serie_2, modal_2]]

#### Clean dataset : double study_uid, if not CT AND PT

In [ ]:
#check double study_uid 
dataset = [x for n, x in enumerate(dataset) if x not in dataset[:n]]
print('dataset cleaned')

In [ ]:
#check Modality 
index = []
for patient in dataset : 
    modal = []
    modal.append(patient[2])
    modal.append(patient[-1])
    if modal[0] == modal[1] : 
        index.append(dataset.index(patient)) #put in index list the index of patient in dataset which have double modality
if len(index) != 0 : #if there is existing index, remove patient from dataset 
    patient_to_remove = []
    for ind in index : 
        patient_to_remove.append(dataset[ind])
    for patient in patient_to_remove : 
        dataset.remove(patient)
print("dataset cleaned")

In [ ]:
print("Number of study after clean :", len(dataset))

#### - Prepare json file to generate nifti in "nifti_builder_from_json.ipynb" script

In [ ]:
study_results = []
for patient in dataset: 
    #patient = [json_path_1, path_serie_1, modal_1, type_1, patient_id, study_uid, json_path_2, path_serie_2, modal_2]
    subliste = []
    subliste.append(patient[1]) #path_1
    subliste.append(patient[2]) #modal 1
    subliste.append(patient[-2]) #path_2
    subliste.append(patient[-1]) #modal_2
    subliste.append(patient[5]) #study_uid
    subliste.append(patient[3]) #type
    subliste.append(patient[4]) #patient id
    study_results.append(subliste)

In [ ]:
#save results as json 
directory = 'abs//path//where to save json results file'
filename = 'name of the result json file'
write_json_file(directory, filename, study_results)